In [1]:
import os
import random
import nltk
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
#from google.colab import drive

nltk.download('punkt_tab')

[nltk_data] Downloading package punkt_tab to /home/gitpod/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [2]:
#drive.mount('/content/drive')
#BASE_DIR = '/content/drive/MyDrive/Colab Notebooks/Corpus10/'
BASE_DIR = "./Corpus10/"
corpus = os.listdir(BASE_DIR)

In [3]:
def leer_texto(path):
    with open(path, "r", encoding="utf8") as entrada:
        return entrada.read()

def limpiar_texto(texto):
    separadores = ["(", ")", ",", ".", ";", ":", "!", "¿", "?", "¡", "[", "]", "{", "}", "<", ">", "-", "_", "\"", "'", "/", "\\", "|", "@", "#", "$", "%", "^", "&", "*", "+", "=", "~", "`"]
    for s in separadores:
        texto = texto.replace(s, " ")
    return texto

def tokenizar(texto):
    puntuacion = '!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~¿¡'
    tokens = nltk.word_tokenize(texto, language="spanish")
    tokens = [t.strip(puntuacion) for t in tokens if t.strip(puntuacion)]
    return nltk.word_tokenize(" ".join(tokens), language="spanish")

def stemmizar(tokens):
    stemmer = nltk.stem.snowball.SpanishStemmer()
    return [stemmer.stem(t) for t in tokens]



In [4]:
archivo_random = random.choice(corpus)
texto = leer_texto(os.path.join(BASE_DIR, archivo_random))
texto = limpiar_texto(texto)
tokens = tokenizar(texto)
print("Tokens de ejemplo:", tokens[:10])

Tokens de ejemplo: ['Copyright', 'C', '2001', 'N', 'K', 'Mayberry', 'A', 'S', 'Kline', 'Don']


In [5]:
indice = {}
for archivo in corpus:
    texto = leer_texto(os.path.join(BASE_DIR, archivo))
    tokens = tokenizar(texto)
    vocabulario = set(tokens)
    for palabra in vocabulario:
        indice.setdefault(palabra, set()).add(archivo)

print(indice.get("Quijote", "No se encontró la palabra"))

{'2000', '9980'}


In [ ]:
indice_stem = {}
for archivo in corpus:
    texto = leer_texto(os.path.join(BASE_DIR, archivo))
    tokens = tokenizar(texto)
    stems = stemmizar(tokens)
    for s in set(stems):
        indice_stem.setdefault(s, set()).add(archivo)

palabra = "oreja"
stem = stemmizar([palabra])[0]
print(f"Búsqueda por stem '{stem}':", indice_stem.get(stem, []))

Búsqueda por stem 'orej': {'8870', '9980', '2000', '7109', '5985', '9895', 'ejemplo_2_PLN.ipynb', '1619'}


In [ ]:
indice_frecuencias = {}
for archivo in corpus:
    texto = leer_texto(os.path.join(BASE_DIR, archivo)).lower()
    tokens = tokenizar(texto)
    for palabra in tokens:
        if palabra not in indice_frecuencias:
            indice_frecuencias[palabra] = {}
        indice_frecuencias[palabra][archivo] = indice_frecuencias[palabra].get(archivo, 0) + 1

tabla_frecuencias = pd.DataFrame(0, index=indice_frecuencias.keys(), columns=corpus)
for palabra, frecs in indice_frecuencias.items():
    for archivo, freq in frecs.items():
        tabla_frecuencias.at[palabra, archivo] = freq

archivo_random = random.choice(corpus)
print(f"Top palabras en '{archivo_random}' (conteo):")
#print(tabla_frecuencias[archivo_random].sort_values(ascending=False).head(20))
tabla_frecuencias[archivo_random].sort_values(ascending=False).head(20)

Top palabras en '2000' (conteo):


,2000
que,20742
de,18409
y,18264
la,10491
a,9873
en,8283
el,8265
no,6338
los,4769
se,4752


In [ ]:
lista_rutas = [os.path.join(BASE_DIR, archivo) for archivo in corpus]
cv = CountVectorizer(input="filename", analyzer="word")
matriz_cv = cv.fit_transform(lista_rutas)

vocabulario_cv = cv.get_feature_names_out()
tabla_cv = pd.DataFrame(
    matriz_cv.toarray(),
    index=corpus,
    columns=vocabulario_cv
)

archivo_random = random.choice(corpus)
print(f"Top palabras con CountVectorizer en '{archivo_random}':")
print(tabla_cv.loc[archivo_random].sort_values(ascending=False).head(20))

Top palabras con CountVectorizer en '9890':
de         2083
la         1034
el          904
en          709
que         542
los         472
del         380
por         345
se          288
su          287
las         278
con         235
al          212
un          177
sus         152
bolívar     145
una         142
para        140
mas         116
como         99
Name: 9890, dtype: int64


In [ ]:
lista_textos = [leer_texto(os.path.join(BASE_DIR, archivo)) for archivo in corpus]
tfidf = TfidfVectorizer()
matriz_tfidf = tfidf.fit_transform(lista_textos)

vocabulario_tfidf = tfidf.get_feature_names_out()
tabla_tfidf = pd.DataFrame(
    matriz_tfidf.toarray(),
    index=corpus,
    columns=vocabulario_tfidf
)

archivo_random = random.choice(corpus)
print(f"Top palabras con TF-IDF en '{archivo_random}':")
print(tabla_tfidf.loc[archivo_random].sort_values(ascending=False).head(20))

Top palabras con TF-IDF en '2000':
que        0.560743
de         0.497052
la         0.283274
en         0.223687
el         0.223147
no         0.171336
los        0.128758
se         0.128299
quijote    0.126976
sancho     0.123180
con        0.115421
por        0.106511
lo         0.094281
las        0.094119
le         0.092337
su         0.091473
don        0.086635
más        0.077979
me         0.068822
dijo       0.068607
Name: 2000, dtype: float64


In [ ]:
corpus = [
    'This is the first document.',
    'This document is the second document.',
    'And this is the third one.',
    'Is this the first document?',
]
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(corpus)

# Convertir a DataFrame para mostrar como tabla
df = pd.DataFrame(X.toarray(), index=corpus, columns=vectorizer.get_feature_names_out())

# Mostrar el DataFrame
df

,and,document,first,is,one,second,the,third,this
This is the first document.,0.000000,0.469791,0.580286,0.384085,0.000000,0.000000,0.384085,0.000000,0.384085
This document is the second document.,0.000000,0.687624,0.000000,0.281089,0.000000,0.538648,0.281089,0.000000,0.281089
And this is the third one.,0.511849,0.000000,0.000000,0.267104,0.511849,0.000000,0.267104,0.511849,0.267104
Is this the first document?,0.000000,0.469791,0.580286,0.384085,0.000000,0.000000,0.384085,0.000000,0.384085
